In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',500)
pd.set_option('display.width',1000)

In [3]:
%%time
data = pd.read_csv('20200221_data.csv')

Wall time: 7.68 s


--------------------------------------------

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS

--------------------------------------

In [5]:
data.head(5)

,CarID,MakeId,Title,Year,Make,Model,Trim,BodyStyle,Rental,Location,GasMileage_city,GasMileage_highway,Price,Mileage,Transmission,FuelType,ExteriorColor,InteriorColor,DoorsNum,MaximumSeating,Engine,Drivetrain,VIN,OptionCount,MajorOptions,Certified,Accident Check,OwnershipHistory,SellingDays,OriginPrice
0,100351603,m1,"2007 Chevrolet Tahoe - $11,999",2007,Chevrolet,Tahoe,NaN,suv,NaN,TX,NaN,NaN,11999.0,129280.0,Automatic,Gasoline,Gold,NaN,NaN,NaN,V8,NaN,1GNFC13J47R120389,0,NaN,NaN,0.0,2.0,1873,12999.0
1,101113873,m1,"2005 Chevrolet Express - $11,995",2005,Chevrolet,Express,NaN,van,NaN,NH,NaN,NaN,11995.0,118113.0,Automatic,NaN,White,NaN,NaN,NaN,NaN,NaN,1GBJG31U251136588,0,NaN,NaN,1.0,1.0,1863,NaN
2,102996725,m1,2010 Chevrolet Silverado 2500HD Work Truck Ext...,2010,Chevrolet,Silverado 2500HD,Work Truck Extended Cab LB 4WD,pickup_truck,NaN,KY,NaN,NaN,15900.0,144000.0,Automatic,Gasoline,White,Gray,4 doors,6 seats,V8,Four-Wheel Drive,1GC5KVBG7AZ218246,2,"Steel Wheels,Tow Package",NaN,0.0,2.0,1837,16900.0
3,103384798,m1,2014 Chevrolet Corvette Stingray 3LT Coupe RWD...,2014,Chevrolet,Corvette,Stingray 3LT Coupe RWD,coupe,NaN,MT,17 MPG City,29 MPG Highway,49995.0,2500.0,7-Speed Manual,Gasoline,Red,Black,2 doors,2 seats,V8,Rear-Wheel Drive,1G1YE2D73E5104797,5,"Bluetooth,Backup Camera,Aluminum Wheels,Naviga...",NaN,0.0,1.0,1832,62900.0
4,104800849,m1,"2002 Chevrolet Monte Carlo SS FWD - $3,440",2002,Chevrolet,Monte Carlo,SS FWD,coupe,NaN,IL,17 MPG City,27 MPG Highway,3440.0,174000.0,Automatic,Gasoline,Black,Black,2 doors,5 seats,V6,Front-Wheel Drive,2G1WX15K829249749,2,"Leather Seats,Aluminum Wheels",NaN,0.0,2.0,1809,4800.0


In [6]:
data.columns

Index(['CarID', 'MakeId', 'Title', 'Year', 'Make', 'Model', 'Trim', 'BodyStyle', 'Rental', 'Location', 'GasMileage_city', 'GasMileage_highway', 'Price', 'Mileage', 'Transmission', 'FuelType', 'ExteriorColor', 'InteriorColor', 'DoorsNum', 'MaximumSeating', 'Engine', 'Drivetrain', 'VIN', 'OptionCount', 'MajorOptions', 'Certified', 'Accident Check', 'OwnershipHistory', 'SellingDays', 'OriginPrice'], dtype='object')

In [11]:
train_data = data.iloc[:,[0,3,4,7,9,12,13,23,26,27,28]]

In [12]:
train_data.isnull().sum()

CarID               0
Year                0
Make                0
BodyStyle           0
Location            0
Price               0
Mileage             0
OptionCount         0
Accident Check      0
OwnershipHistory    0
SellingDays         0
dtype: int64

In [13]:
train_data.dtypes

CarID                 int64
Year                  int64
Make                 object
BodyStyle            object
Location             object
Price               float64
Mileage             float64
OptionCount           int64
Accident Check      float64
OwnershipHistory    float64
SellingDays           int64
dtype: object

In [15]:
locations = pd.get_dummies(train_data.iloc[:,4])

In [16]:
train_data = train_data.drop(columns=['Location'])

In [25]:
train_data = pd.concat([train_data, locations], axis=1)

In [26]:
train_data.head()

,CarID,Year,Make,BodyStyle,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,ON,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,100351603,2007,Chevrolet,suv,11999.0,129280.0,0,0.0,2.0,1873,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,101113873,2005,Chevrolet,van,11995.0,118113.0,0,1.0,1.0,1863,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,102996725,2010,Chevrolet,pickup_truck,15900.0,144000.0,2,0.0,2.0,1837,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,103384798,2014,Chevrolet,coupe,49995.0,2500.0,5,0.0,1.0,1832,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,104800849,2002,Chevrolet,coupe,3440.0,174000.0,2,0.0,2.0,1809,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
train_data.shape

(2151338, 60)

In [28]:
train_data.isnull().sum()

CarID               0
Year                0
Make                0
BodyStyle           0
Price               0
Mileage             0
OptionCount         0
Accident Check      0
OwnershipHistory    0
SellingDays         0
AL                  0
AR                  0
AZ                  0
CA                  0
CO                  0
CT                  0
DC                  0
DE                  0
FL                  0
GA                  0
IA                  0
ID                  0
IL                  0
IN                  0
KS                  0
KY                  0
LA                  0
MA                  0
MD                  0
ME                  0
MI                  0
MN                  0
MO                  0
MS                  0
MT                  0
NC                  0
ND                  0
NE                  0
NH                  0
NJ                  0
NM                  0
NV                  0
NY                  0
OH                  0
OK                  0
ON        

-------------------------------------------------------------

In [32]:
part = train_data.sample(frac=0.005)

In [34]:
data_for_train = part.drop(columns=['CarID', 'Make', 'BodyStyle'])

In [35]:
data_for_train.shape

(10757, 57)

------------------------

In [36]:
mms = MinMaxScaler()

In [38]:
data_for_train

,Year,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,ON,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
1154825,2016,21998.0,40229.0,7,0.0,2.0,41,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1004330,2010,12995.0,112250.0,0,1.0,2.0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1089764,2015,41688.0,106960.0,26,0.0,1.0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1821553,2017,21700.0,30378.0,2,0.0,1.0,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1073523,2017,25992.0,35181.0,5,1.0,2.0,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857625,2011,10299.0,99924.0,4,1.0,2.0,84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1527383,2017,44990.0,32902.0,7,0.0,2.0,104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
524944,2010,11500.0,183376.0,0,0.0,3.0,414,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1927793,2019,24851.0,8684.0,6,0.0,1.0,7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
data_for_train.iloc[:,[0,1,2,3,4,5,6]]

,Year,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays
1154825,2016,21998.0,40229.0,7,0.0,2.0,41
1004330,2010,12995.0,112250.0,0,1.0,2.0,8
1089764,2015,41688.0,106960.0,26,0.0,1.0,15
1821553,2017,21700.0,30378.0,2,0.0,1.0,42
1073523,2017,25992.0,35181.0,5,1.0,2.0,32
...,...,...,...,...,...,...,...
857625,2011,10299.0,99924.0,4,1.0,2.0,84
1527383,2017,44990.0,32902.0,7,0.0,2.0,104
524944,2010,11500.0,183376.0,0,0.0,3.0,414
1927793,2019,24851.0,8684.0,6,0.0,1.0,7


In [40]:
%%time
data_for_train.iloc[:,[0,1,2,3,4,5,6]] = mms.fit_transform(data_for_train.iloc[:,[0,1,2,3,4,5,6]])

Wall time: 17.1 ms


In [41]:
data_for_train.iloc[:,[0,1,2,3,4,5,6]]

,Year,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays
1154825,0.894737,0.056722,0.114045,0.212121,0.0,0.071429,0.024924
1004330,0.736842,0.032658,0.318497,0.000000,0.2,0.071429,0.004863
1089764,0.868421,0.109352,0.303480,0.787879,0.0,0.000000,0.009119
1821553,0.921053,0.055925,0.086080,0.060606,0.0,0.000000,0.025532
1073523,0.921053,0.067398,0.099715,0.151515,0.2,0.071429,0.019453
...,...,...,...,...,...,...,...
857625,0.763158,0.025452,0.283506,0.121212,0.2,0.071429,0.051064
1527383,0.921053,0.118178,0.093245,0.212121,0.0,0.071429,0.063222
524944,0.736842,0.028662,0.520408,0.000000,0.0,0.142857,0.251672
1927793,0.973684,0.064348,0.024496,0.181818,0.0,0.000000,0.004255


In [45]:
data_for_train.iloc[:,7:] = data_for_train.iloc[:,7:]*0.5

In [47]:
data_for_train = data_for_train.astype('float32')

In [48]:
db1 = DBSCAN(eps=0.2,
    min_samples=5,
    metric='euclidean',
    n_jobs=6)

In [49]:
%%time
y_db1 = db1.fit_predict(data_for_train)

Wall time: 1.91 s


In [50]:
np.unique(y_db1)

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101],
      dtype=int64)

In [51]:
unique_db1, counts_db1 = np.unique(y_db1, return_counts=True)

In [52]:
dict(zip(unique_db1, counts_db1))

{-1: 564,
 0: 274,
 1: 217,
 2: 116,
 3: 65,
 4: 378,
 5: 764,
 6: 127,
 7: 422,
 8: 192,
 9: 538,
 10: 52,
 11: 363,
 12: 136,
 13: 208,
 14: 128,
 15: 840,
 16: 100,
 17: 190,
 18: 124,
 19: 294,
 20: 176,
 21: 200,
 22: 42,
 23: 296,
 24: 89,
 25: 234,
 26: 256,
 27: 74,
 28: 314,
 29: 216,
 30: 27,
 31: 64,
 32: 118,
 33: 55,
 34: 81,
 35: 54,
 36: 15,
 37: 49,
 38: 178,
 39: 78,
 40: 199,
 41: 31,
 42: 104,
 43: 96,
 44: 47,
 45: 119,
 46: 35,
 47: 32,
 48: 187,
 49: 56,
 50: 73,
 51: 69,
 52: 32,
 53: 69,
 54: 12,
 55: 14,
 56: 27,
 57: 6,
 58: 7,
 59: 60,
 60: 95,
 61: 5,
 62: 16,
 63: 20,
 64: 29,
 65: 24,
 66: 41,
 67: 28,
 68: 25,
 69: 9,
 70: 12,
 71: 18,
 72: 8,
 73: 13,
 74: 28,
 75: 6,
 76: 22,
 77: 36,
 78: 13,
 79: 22,
 80: 12,
 81: 50,
 82: 15,
 83: 13,
 84: 19,
 85: 31,
 86: 23,
 87: 9,
 88: 5,
 89: 30,
 90: 5,
 91: 22,
 92: 5,
 93: 5,
 94: 10,
 95: 6,
 96: 5,
 97: 7,
 98: 8,
 99: 8,
 100: 11,
 101: 5}

In [58]:
db2 = DBSCAN(eps=0.3,
    min_samples=5,
    metric='euclidean',
    n_jobs=6)

In [59]:
%%time
y_db2 = db2.fit_predict(data_for_train)

Wall time: 2.53 s


In [60]:
np.unique(y_db2)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47],
      dtype=int64)

In [61]:
unique_db2, counts_db2 = np.unique(y_db2, return_counts=True)

In [62]:
dict(zip(unique_db2, counts_db2))

{-1: 122,
 0: 369,
 1: 36,
 2: 261,
 3: 908,
 4: 80,
 5: 485,
 6: 160,
 7: 500,
 8: 223,
 9: 712,
 10: 56,
 11: 416,
 12: 176,
 13: 267,
 14: 947,
 15: 128,
 16: 229,
 17: 142,
 18: 375,
 19: 199,
 20: 236,
 21: 50,
 22: 370,
 23: 291,
 24: 55,
 25: 327,
 26: 413,
 27: 259,
 28: 81,
 29: 135,
 30: 96,
 31: 64,
 32: 123,
 33: 213,
 34: 92,
 35: 244,
 36: 34,
 37: 126,
 38: 158,
 39: 39,
 40: 222,
 41: 82,
 42: 117,
 43: 34,
 44: 13,
 45: 35,
 46: 28,
 47: 29}

In [63]:
db3 = DBSCAN(eps=0.5,
    min_samples=5,
    metric='euclidean',
    n_jobs=6)

In [64]:
%%time
y_db3 = db3.fit_predict(data_for_train)

Wall time: 2.58 s


In [65]:
np.unique(y_db3)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47],
      dtype=int64)

In [66]:
unique_db3, counts_db3 = np.unique(y_db3, return_counts=True)

In [67]:
dict(zip(unique_db3, counts_db3))

{-1: 15,
 0: 381,
 1: 36,
 2: 261,
 3: 261,
 4: 912,
 5: 82,
 6: 488,
 7: 162,
 8: 505,
 9: 224,
 10: 715,
 11: 57,
 12: 419,
 13: 178,
 14: 270,
 15: 949,
 16: 128,
 17: 232,
 18: 146,
 19: 376,
 20: 199,
 21: 238,
 22: 52,
 23: 371,
 24: 298,
 25: 57,
 26: 332,
 27: 416,
 28: 82,
 29: 137,
 30: 97,
 31: 67,
 32: 124,
 33: 214,
 34: 92,
 35: 246,
 36: 38,
 37: 127,
 38: 159,
 39: 121,
 40: 40,
 41: 225,
 42: 83,
 43: 34,
 44: 13,
 45: 36,
 46: 29,
 47: 33}

In [68]:
part.shape

(10757, 60)

In [69]:
len(y_db3)

10757

In [70]:
part['db3_labels'] = y_db3

In [71]:
part[part['db3_labels'] == 11]

,CarID,Year,Make,BodyStyle,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,ON,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,db3_labels
464977,248955417,2019,Ram,pickup_truck,38499.0,13098.0,5,0.0,1.0,138,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11
1995835,255719467,2018,Toyota,crossover,20388.0,14522.0,4,0.0,1.0,69,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11
1820428,256582677,2016,Volvo,sedan,14959.0,81095.0,9,0.0,2.0,55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11
368151,259539030,2015,Nissan,crossover,11908.0,94301.0,1,0.0,1.0,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11
30343,247620004,2018,Chevrolet,sedan,12853.0,34256.0,4,0.0,1.0,154,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11
1059833,257104689,2015,GMC,suv,29900.0,107891.0,6,0.0,1.0,51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11
125858,257373325,2016,Chevrolet,crossover,12442.0,78275.0,4,0.0,1.0,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11
563387,249367960,2018,Ford,coupe,21200.0,20175.0,3,0.0,1.0,135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11
24326,245867908,2016,Chevrolet,crossover,17887.0,84725.0,5,0.0,2.0,172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11
1102763,261311993,2007,GMC,crossover,9999.0,115099.0,0,0.0,2.0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11
